In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import models, transforms
from torchvision.transforms import InterpolationMode
import numpy as np
from typing import Tuple


In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [49]:
class TaskDataset(Dataset):
    def __init__(self, transform=None):
        self.ids = []
        self.imgs = []
        self.labels = []
        self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)


In [50]:
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5)
])


val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])


In [51]:
import __main__
torch.serialization.add_safe_globals([__main__.TaskDataset])

dataset = torch.load("Train.pt", weights_only=False)
print(f"Loaded dataset of length: {len(dataset)}")

# Split dataset 90/10
dataset_size = len(dataset)
val_size = int(0.1 * dataset_size)
train_size = dataset_size - val_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Assign transforms to base datasets inside subsets
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform


Loaded dataset of length: 100000


In [52]:
class ImgLabelDataset(Dataset):
    def __init__(self, base_dataset):
        self.base = base_dataset

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        sample = self.base[idx]
        return sample[1], sample[2]


In [53]:
batch_size = 128  # Adjust based on your GPU memory

train_loader = DataLoader(ImgLabelDataset(train_dataset), batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(ImgLabelDataset(val_dataset), batch_size=batch_size, shuffle=False, num_workers=0)


In [54]:
from torchvision import models
import torch.nn as nn

model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 10)  # 10 classes for your task
model = model.to(device)



In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def pgd_attack(model, images, labels, eps, alpha, iters, device='cuda'):
    model.eval()
    images, labels = images.to(device), labels.to(device)
    ori_images = images.clone().detach()
    perturbed_images = images + torch.empty_like(images).uniform_(-eps, eps)
    perturbed_images = torch.clamp(perturbed_images, 0, 1).detach()
    loss_fn = nn.CrossEntropyLoss()
    for _ in range(iters):
        perturbed_images.requires_grad = True
        outputs = model(perturbed_images)
        loss = loss_fn(outputs, labels)
        model.zero_grad()
        loss.backward()
        grad_sign = perturbed_images.grad.sign()
        adv_images = perturbed_images + alpha * grad_sign
        eta = torch.clamp(adv_images - ori_images, -eps, eps)
        perturbed_images = torch.clamp(ori_images + eta, 0, 1).detach()
    model.train()
    return perturbed_images

In [ ]:
def train_one_epoch_clean(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

In [ ]:
def train_one_epoch_adv(model, loader, optimizer, criterion, device,
                        eps=4/255, alpha=1/255, pgd_iters=7, mix_clean=True):
    model.train()
    running_loss = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        adv_imgs = pgd_attack(model, imgs, labels, eps, alpha, pgd_iters, device)
        optimizer.zero_grad()
        if mix_clean:
            out_clean = model(imgs)
            out_adv = model(adv_imgs)
            loss = 0.5 * (criterion(out_clean, labels) + criterion(out_adv, labels))
        else:
            out_adv = model(adv_imgs)
            loss = criterion(out_adv, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

In [ ]:
def validate(model, loader, device, attack=None, eps=4/255, alpha=1/255, iters=7):
    model.eval()
    correct_clean = 0
    correct_adv = 0
    total = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        total += labels.size(0)
        outputs = model(imgs)
        _, pred = outputs.max(1)
        correct_clean += pred.eq(labels).sum().item()
        if attack is not None:
            with torch.enable_grad():
                adv_imgs = attack(model, imgs, labels, eps, alpha, iters, device=device)
            outputs_adv = model(adv_imgs)
            _, pred_adv = outputs_adv.max(1)
            correct_adv += pred_adv.eq(labels).sum().item()
    clean_acc = 100 * correct_clean / total
    adv_acc = 100 * correct_adv / total if attack is not None else None
    return clean_acc, adv_acc

In [ ]:
# ---- TRAINING FLOW ----
best_val_acc = 0
best_model_path = "submission_model01.pt"

In [ ]:
# Phase 1: Clean training (e.g., 40 epochs)
for epoch in range(40):
    print(f"Epoch {epoch+1}/40 - Clean training")
    loss = train_one_epoch_clean(model, train_loader, optimizer, criterion, device)
    print(f"Loss: {loss:.4f}")
    val_acc, _ = validate(model, val_loader, device)
    print(f"Validation Accuracy: {val_acc:.2f}%")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved best clean model with accuracy {best_val_acc:.2f}%")

Epoch 1/60 - Clean training
Train batch 0/704 loss: 2.4518
Train batch 100/704 loss: 1.7031
Train batch 200/704 loss: 1.5539
Train batch 300/704 loss: 1.5417
Train batch 400/704 loss: 1.5765
Train batch 500/704 loss: 1.4241
Train batch 600/704 loss: 1.5569
Train batch 700/704 loss: 1.3850
Clean training epoch loss: 1.5601
Loss: 1.5601
Validation Accuracy: 54.360%
Saved best clean model with accuracy 54.360%
Epoch 2/60 - Clean training
Train batch 0/704 loss: 1.4592
Train batch 100/704 loss: 1.5436
Train batch 200/704 loss: 1.3399
Train batch 300/704 loss: 1.4240
Train batch 400/704 loss: 1.4923
Train batch 500/704 loss: 1.4721
Train batch 600/704 loss: 1.5403
Train batch 700/704 loss: 1.4344
Clean training epoch loss: 1.4640
Loss: 1.4640
Validation Accuracy: 54.900%
Saved best clean model with accuracy 54.900%
Epoch 3/60 - Clean training
Train batch 0/704 loss: 1.3535
Train batch 100/704 loss: 1.3910
Train batch 200/704 loss: 1.3294
Train batch 300/704 loss: 1.4055
Train batch 400/704 

In [63]:
# Load best clean model before adversarial fine-tuning
model.load_state_dict(torch.load(best_model_path))

# Lower LR for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=5e-5)

In [72]:
best_adv_val_acc = 0.0
best_adv_model_path = "best_adv_model4.pt"
num_adv_epochs = 15

for epoch in range(num_adv_epochs):
    print(f"Epoch {epoch+1}/{num_adv_epochs} - Adversarial fine-tuning")
    adv_loss = train_one_epoch_adv(model, train_loader, optimizer, criterion, device,
                                  eps=4/255, alpha=1/255, pgd_iters=7, mix_clean=True)
    print(f"Adv Train Loss: {adv_loss:.4f}")
    
    val_acc, adv_acc = validate(model, val_loader, device, attack=pgd_attack,
                               eps=4/255, alpha=1/255, iters=7)
    print(f"Validation Clean Accuracy: {val_acc:.2f}%")
    print(f"Validation Adversarial Accuracy: {adv_acc:.2f}%")
    
    # Save checkpoint if adversarial accuracy improved
    if adv_acc > best_adv_val_acc:
        best_adv_val_acc = adv_acc
        torch.save(model.state_dict(), best_adv_model_path)
        print(f"Saved best adversarial fine-tuned model at epoch {epoch+1} with adv accuracy {adv_acc:.2f}%")


Epoch 1/15 - Adversarial fine-tuning
Adversarial training epoch loss: 1.7598
Adv Train Loss: 1.7598
Validation Clean Accuracy: 57.83%
Validation Adversarial Accuracy: 68.58%
Saved best adversarial fine-tuned model at epoch 1 with adv accuracy 68.58%
Epoch 2/15 - Adversarial fine-tuning
Adversarial training epoch loss: 1.7913
Adv Train Loss: 1.7913
Validation Clean Accuracy: 57.78%
Validation Adversarial Accuracy: 69.08%
Saved best adversarial fine-tuned model at epoch 2 with adv accuracy 69.08%
Epoch 3/15 - Adversarial fine-tuning
Adversarial training epoch loss: 1.7523
Adv Train Loss: 1.7523
Validation Clean Accuracy: 57.82%
Validation Adversarial Accuracy: 68.86%
Epoch 4/15 - Adversarial fine-tuning
Adversarial training epoch loss: 1.7696
Adv Train Loss: 1.7696
Validation Clean Accuracy: 57.47%
Validation Adversarial Accuracy: 68.71%
Epoch 5/15 - Adversarial fine-tuning
Adversarial training epoch loss: 1.7191
Adv Train Loss: 1.7191
Validation Clean Accuracy: 57.38%
Validation Adversa

In [75]:

# Save final model for submission
print("Saving final model for submission...")
torch.save(model.state_dict(), "final_submission_model4.pt")

Saving final model for submission...


In [ ]:
import requests

token = "53077688"  # your token here

with open("submission_model.pt", "rb") as f:
    response = requests.post(
        "http://34.122.51.94:9090/robustness",
        files={"file": f},
        headers={"token": token, "model-name": "resnet18"}
    )


print(response.json())


{'clean_accuracy': 0.6093333333333333, 'fgsm_accuracy': 0.123, 'pgd_accuracy': 0.005}


In [69]:
print(response.json())

{'detail': "Error during evaluation, e=HTTPException(status_code=400, detail='Model clean accuracy is too low.')"}


c:\Users\maitr\miniconda3\envs\tf\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
{'clean_accuracy': 0.563, 'fgsm_accuracy': 0.301, 'pgd_accuracy': 0.26266666666666666}